# 1 Script setup

In [1]:
import pandas as pd
import pandasql as ps
import numpy as np
import os
os.chdir(r'C:\Users\merom\Documents\GitHub\collancer\investment-nsw\output\tenders')

# 2. Consolidate procurements

In [2]:
tables = [pd.read_excel(x,header=2) for x in os.listdir()]
data = pd.concat(tables,axis=0).drop_duplicates()

# 3. Clean columns

In [4]:
data['Estimated amount payable to the contractor (including GST)'] = data['Estimated amount payable to the contractor (including GST)'].apply(lambda x: float(str(x).replace('$','').replace(',','')))
data['contract_start'] = pd.to_datetime(data['Contract Period'].apply(lambda x: x.split(' to ')[0]),errors = 'coerce')
data['contract_end'] = pd.to_datetime(data['Contract Period'].apply(lambda x: x.split(' to ')[1]),errors = 'coerce')
data = data[(data.contract_start.isnull()==False)&(data.contract_end.isnull()==False)].reset_index(drop=True)
data['contract_length'] = (data['contract_end']-data['contract_start']).astype(np.int64)/(1000000000*60*60*24)

# 4. Create calendar year frame

In [5]:
frame = pd.DataFrame()
for i in range(2000,2025):
    temp_table = pd.DataFrame(data = [[f'1-Jan-{i}',f'31-Dec-{i}']],columns = ('year_start','year_end'))
#    temp_table['year_start']= f'1-Jan-{i}'
#    temp_table['year_end']= f'31-Dec-{i}'
    frame = pd.concat([frame,temp_table])
frame = frame.reset_index(drop=True)
frame.year_start = pd.to_datetime(frame.year_start)
frame.year_end = pd.to_datetime(frame.year_end)

# 5. Combine dataframe and aportion contract value to year

In [6]:
sqlcode = '''
select A.*,B.*
from frame A
inner join data B 
on (A.year_start <= B.contract_end) and (A.year_end >= b.contract_start)
'''
# Merge dataframes
newdf = ps.sqldf(sqlcode,locals())

#For each row Identify relevant start date and end date
newdf['row_time_start'] = pd.to_datetime(newdf[['year_start','contract_start']].apply(max,axis=1))
newdf['row_time_end'] = pd.to_datetime(newdf[['year_end','contract_end']].apply(min,axis=1))

newdf['attributable_days'] = (newdf['row_time_end']-newdf['row_time_start']).astype(np.int64)/(1000000000*60*60*24)
newdf['attributable_share'] = (newdf['attributable_days']/newdf['contract_length']).apply(lambda x: 1 if x>1 else x)
newdf['attributable_cost']=newdf['attributable_share']*newdf['Estimated amount payable to the contractor (including GST)']


# 6. Change working directory to output folder

In [8]:
os.chdir(r'C:\Users\merom\Documents\GitHub\collancer\investment-nsw\output')

In [9]:
newdf[['year_start','year_end','Category','attributable_cost']].groupby(['year_start','year_end','Category']).sum().to_csv('nsw_procurement.csv')

In [16]:
newdf.loc[newdf.year_start>='2022-12-31',['year_start','year_end','Category','attributable_cost']].groupby(['year_start','year_end','Category']).sum().to_csv('nsw_procurement2324.csv')

In [19]:
newdf.loc[(newdf.year_start>='2022-12-31')&(newdf['CAN ID']=='SR4057427954/RA454'),:]

,year_start,year_end,Contract Title,CAN ID,Agency,Category,Particulars of the goods or services to be provided under this contract,Published,Contract Period,Estimated amount payable to the contractor (including GST),...,Contractor Name,Last Updated,contract_start,contract_end,contract_length,row_time_start,row_time_end,attributable_days,attributable_share,attributable_cost
32933,2023-01-01 00:00:00.000000,2023-12-31 00:00:00.000000,Credits Supply Fund - Reverse auctions for in-...,SR4057427954/RA454,Biodiversity Conservation & Science,Other\t,Purchase of biodiversity credits for resale t...,31-May-2024,6-Dec-2023 to 7-Jun-2024,352000.0,...,Department of Planning and Environment,31-May-2024 4:19pm,2023-12-06 00:00:00.000000,2024-06-07 00:00:00.000000,184.0,2023-12-06,2023-12-31,25.0,0.135870,47826.086957
46855,2024-01-01 00:00:00.000000,2024-12-31 00:00:00.000000,Credits Supply Fund - Reverse auctions for in-...,SR4057427954/RA454,Biodiversity Conservation & Science,Other\t,Purchase of biodiversity credits for resale t...,31-May-2024,6-Dec-2023 to 7-Jun-2024,352000.0,...,Department of Planning and Environment,31-May-2024 4:19pm,2023-12-06 00:00:00.000000,2024-06-07 00:00:00.000000,184.0,2024-01-01,2024-06-07,158.0,0.858696,302260.869565
